In [ ]:
import pandas as pd
import numpy as np
import os, json, sys

# Data Loading

In [ ]:
farm_april_aug = pd.read_excel('data/Daily_Data_AprilAug.xlsx', sheet_name='Sheet1')
farm_aug_dec = pd.read_excel('data/Daily_Data_AugDec.xlsx', sheet_name='Sheet1')
env_data = pd.read_csv('data/Environmental_data.csv')
repro_data = pd.read_excel('data/ReproAllCows.XLS', sheet_name='Sheet1')

### Processing AfiFarm Data, Combining April - Dec

In [ ]:
feat = ['Date', 'ID', 'yeild', 'conductivity', 'fat',
        'protein', 'lactose', 'scc', 'steps_per_hour', 
        'rest_time', 'rest_bout', 'rest_ratio', 'restlessness', 
        'rest_per_bout']

In [ ]:
feat1 = ['Date', 'Animal_ID', 'Yield (gr)', 'Conductivity', 'Fat (%)', 'Protein (%)', 'Lactose (%)', 'SCC (*1000/ml)',
 'Activity (steps/hour)', 'RestTime (minutes)', 'RestBout (#)', 'RestRatio (%)', 'RestRestlessness', 'RestPerBout (minutes)']
farm_april_aug = farm_april_aug[feat1]

In [ ]:
feat2 = ['Date', 'Animal_ID', 'Yield(gr)', 'Conductivity', 'Fat(%)', 'Protein(%)', 'Lactose(%)', 'Avg_SCC(*1000/ml)',
 'Activity(steps/hour)', 'RestTime(min)', 'RestBout', 'RestRatio(%)', 'RestRestlessness', 'RestPerBout(min)']
farm_aug_dec = farm_aug_dec[feat2]

In [ ]:
farm_april_aug.columns = feat
farm_aug_dec.columns = feat

In [ ]:
print(farm_april_aug.Date.iloc[0])
print(farm_aug_dec.Date.iloc[0])

In [ ]:
temp = farm_april_aug[['ID', 'Date']].drop_duplicates()
farm_april_aug = pd.merge(temp, farm_april_aug, left_on=['ID', 'Date'], right_on=['ID', 'Date'])

temp = farm_aug_dec[['ID', 'Date']].drop_duplicates()
farm_aug_dec = pd.merge(temp, farm_aug_dec, left_on=['ID', 'Date'], right_on=['ID', 'Date'])

In [ ]:
'''
Steps:
    1. drop null columns
    2. convert date to correct format
'''
def dateToInt1(x):
    if '-' in str(x):
        dmy = str(x).split(' ')[0].split('-')
        return int(dmy[0])*365 + int(dmy[1])*30+int(dmy[2])
    else:
        dmy = str(x).split(' ')[0].split('/')
        return int(dmy[2])*365 + int(dmy[0])*30+int(dmy[1])


farm_april_aug = farm_april_aug.dropna()
farm_aug_dec = farm_aug_dec.dropna()

farm_april_aug['Date'] = farm_april_aug.Date.apply(lambda x: dateToInt1(x))
farm_aug_dec['Date'] = farm_aug_dec.Date.apply(lambda x: dateToInt1(x))

In [ ]:
farm_data = pd.concat([farm_april_aug, farm_aug_dec])

In [ ]:
farm_data1 = farm_data.copy()
farm_data1.head()

### Processing Environmental Data

In [ ]:
'''
Taking only columns H, I, J from the Environmental_data.csv
'''
env_data = env_data[['Date',
                     'Temperature (S-THB 20427137:20353090-1), *C, RX "3" Pen 1',
                     'RH (S-THB 20427137:20353090-2), %, RX "3" Pen 1',
                     'Dew Point (S-THB 20427137:20353090-3), *C, RX "3" Pen 1']]
env_data.columns = ['Date','temp','rh','dew_point']
env_data.head()

In [ ]:
'''
Steps:
    1. Removing rows with null values
    2. Removing hours:minutes from the Date column,
    3. Averaging out the temp, rh, dew_point for each day and storing 1 row per day

'''

# Step 1
env_data1 = env_data.copy()
env_data1 = env_data1.dropna()

# Step 2
env_data1['Date'] = env_data1.Date.apply(lambda x: x.split(' ')[0])

# Step 3
env_data1 = env_data1.groupby(['Date'], as_index=False).mean()

In [ ]:
def dateToInt1(x):
    dmy = x.split('/')
    return (int(dmy[2]) + 2000)*365 + int(dmy[0])*30 + int(dmy[1])

env_data1['Date'] = env_data1.Date.apply(lambda x: dateToInt1(x))
env_data1.head()

### Preprocessing Repro All Cows 

In [ ]:
'''
Steps:
    1. Extracting required columns 
    2. Removing rows with null values
'''

repro_data1 = repro_data.copy()
repro_data1 = repro_data1[['ID', 'FDAT', 'LACT', 'DATB1', 'DIMB1', 'OUTB1']]
repro_data1 = repro_data1.dropna()
repro_data1.head()

In [ ]:
'''
    Removing the rows where OUTB1 =='.' and not 'P' or 'O'
'''

repro_data1 = repro_data1[repro_data1['OUTB1']!='.']

In [ ]:
'''
    Converting date to integer
'''

def dateToInt2(x):
    dmy = str(x).split(' ')[0].split('-')
    return int(dmy[0])*365 + int(dmy[1])*30+int(dmy[2])

repro_data1['FDAT'] = repro_data1['FDAT'].apply(lambda x: dateToInt2(x))
repro_data1['DATB1'] = repro_data1['DATB1'].apply(lambda x: dateToInt2(x))
repro_data1.head()

# Combining Environment Data with ReproAllCows

In [ ]:
data = pd.merge(repro_data1, env_data1, left_on='DATB1', right_on='Date')

In [ ]:
data.count()

# Combining Env, ReproAllCows, with AfiFarm

In [ ]:
data1 = pd.merge(farm_data1, data, left_on=['ID', 'Date'], right_on=['ID', 'Date'])

In [ ]:
data1.head()

# Train model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1 - Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
'''
data needs to be loaded from true data not dummy
'''
y = data1[['OUTB1']]
x = data1[['LACT', 'DIMB1', 'temp', 'rh', 'dew_point',
          'yeild', 'conductivity', 'fat', 'restlessness',
          'protein', 'lactose', 'scc', 'steps_per_hour', 
          'rest_time', 'rest_bout', 'rest_ratio', 
          'rest_per_bout']]


xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=0.2)

for i in [10,100,1000]:
    for j in [3, 4, 5]:
        clf = RandomForestClassifier(n_estimators=i, max_depth=j, random_state=42)
        clf.fit(xTrain,yTrain)
        score1 = clf.score(xTest, yTest)
        print(i, j, score1)

## 2 - Logistic Regression

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

scaler = MinMaxScaler()

y = data1[['OUTB1']]
x = data1[['LACT', 'DIMB1', 'temp', 'rh', 'dew_point',
          'yeild', 'conductivity', 'fat', 'restlessness',
          'protein', 'lactose', 'scc', 'steps_per_hour', 
          'rest_time', 'rest_bout', 'rest_ratio', 
          'rest_per_bout']]

scaled_data = scaler.fit_transform(x)

xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=0.2)

In [ ]:
clf = LogisticRegression(random_state=0, solver='liblinear').fit(xTrain, yTrain)
score1 = clf.score(xTest, yTest)

In [ ]:
score1